In [2]:


from dotenv import load_dotenv
load_dotenv(dotenv_path=".env.local")
import os 
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec



c:\Users\sgale\miniconda3\envs\rateMyProfessor\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
# pc.create_index(name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec("aws", "us-east-1"))

In [4]:
import json 
data = json.load(open("reviews.json"))
data['reviews']
len(data['reviews'])

20

In [6]:
processed_data = [] 
client = OpenAI(
        api_key= os.getenv("OPENAI_API_KEY")
)

for review in data['reviews']:
    summary = f'Professor ${review["professor"]} who teaches "${review["subject"]}" with an overall rating of "${review["stars"]}" stars. Sample reviews: "${review["review"]}"'
    
    response = client.embeddings.create(
        input=summary,
        model='text-embedding-3-small'
    )
    embeddings = response.data[0].embedding 
    processed_data.append({ 
        "values" : embeddings, 
        "id": review["professor"], 
        "metadata" : {
            "professorName": review["professor"],
            "reviews" : review["review"],
            "subject" : review["subject"],
            "stars" : review["stars"] 
        }
        })

In [7]:
index = pc.Index('rag')
index.upsert( 
            vectors=processed_data, 
            namespace="ns1"
             )

{'upserted_count': 20}

In [7]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}

: 